In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import wikipedia
from googlesearch import search
import re
import requests
import random
import time
from itertools import chain

In [3]:
def TAE(expr): # Trial and Error Module
    try:
        var = eval(expr)
    except:
        var = None
    return var

In [4]:
def key_financials(sub_soup, var):
    if sub_soup!=None:
        lis = sub_soup.table.find_all('tr')
        notyear = []
        year = []
        if var != 'Accounts':
            for j in lis:
                if j.find('th').text == var:
                    for i in lis[lis.index(j)].find_all('td'):
                        notyear.append(i.text)
            return notyear
        else:
            for j in lis:
                if j.find('th').text == 'Accounts':
                    for i in lis[lis.index(j)].find_all('time'):
                        year.append(i.text)
            return year
    else:
        return None

In [136]:
# Pending Task: Apply Failsafe
def CompanyCheck(ID, soup):
    if soup!=None:
        try:
            # Non returned variables
            a1 = TAE('''soup.find_all('div', class_ = 'Summary__detail')''')

            # MidSupport Variables
            if len(a1) > 0:
                Summary__detail = []
                for i in range(6):
                    Summary__detail.append((list(a1[i].stripped_strings))[1])
            else:
                Summary__detail = [None] * 6

            sub_soup = TAE('''soup.find('div', attrs = {'id': 'key-financials'})''')
            financials = {
                'accounts' : key_financials(sub_soup, 'Accounts'),
                'cash' : key_financials(sub_soup, 'Cash'),
                'net_worth' : key_financials(sub_soup, 'Net Worth'),
                'total_assets' : key_financials(sub_soup, 'Total Current Assets'),
                'total_liabilities': key_financials(sub_soup, 'Total Current Liabilities')
            }

            # Returned Variables
            company_name = Summary__detail[0]
            company_type = Summary__detail[1]
            company_status = Summary__detail[2]
            incorporated_on = Summary__detail[3]
            SIC_detail = Summary__detail[4]
            accounts = Summary__detail[5]
            
            vls = list(financials.values())
            if not any(vls) == False:
                accounts_year = financials['accounts'][-1]
                cash = financials['cash'][-1]
                net_worth = financials['net_worth'][-1]
                total_assets = financials['total_assets'][-1]
                total_liabilities = financials['total_liabilities'][-1]
            else:
                accounts_year = None
                cash = None
                net_worth = None
                total_assets = None
                total_liabilities = None            

            # EndTransform Variables
            data = {
                'ID':ID,
                'company_name':company_name,
                'company_type':company_type,
                'company_status':company_status,
                'incorporated_on':incorporated_on,
                'SIC_detail':SIC_detail,
                'accounts':accounts,
                'accounts_year':accounts_year,
                'cash':cash,
                'net_worth':net_worth,
                'total_assets':total_assets,
                'total_liabilities':total_liabilities
            }

        except (IndexError, KeyError):
                    data = {
                        'ID':ID,
                        'company_name':None,
                        'company_type':None,
                        'company_status':None,
                        'incorporated_on':None,
                        'SIC_detail':None,
                        'accounts':None,
                        'accounts_year':None,
                        'cash':None,
                        'net_worth':None,
                        'total_assets':None,
                        'total_liabilities':None
                    }
    else:
        data = {
            'ID':ID,
            'company_name':None,
            'company_type':None,
            'company_status':None,
            'incorporated_on':None,
            'SIC_detail':None,
            'accounts':None,
            'accounts_year':None,
            'cash':None,
            'net_worth':None,
            'total_assets':None,
            'total_liabilities':None
        }
    return data

In [82]:
def get_soup(company):
    URL = 'https://companycheck.co.uk/company/'+company
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
    referer = 'https://www.google.com'
    headers = {'user-agent': user_agent, 'referer':referer}
    r = requests.get(URL, timeout= 30, allow_redirects=True, headers = headers)
    if r.status_code == 200:
        pass
    else:
        print('Connection --> ',r.status_code)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup

In [83]:
filepath = '/Users/mayankmahawar/Documents/GitHub/WebScrap/Book1.xlsx'
df = pd.read_excel(filepath,sheet_name = 'Sheet1')
companies = list(df['Company Number.1'])

In [ ]:
import xlsxwriter 
  
workbook = xlsxwriter.Workbook('Data1.xlsx') 
worksheet = workbook.get_sheet(0)
  
# Start from the first cell. 
# Rows and columns are zero indexed. 
row = 0
column = 0
content = list(result.values())
# iterating through content list 
 for 
    for item in content : 

        # write operation perform 
        worksheet.write(row, column, item) 

        # incrementing the value of row by one 
        # with each iteratons. 
        column += 1
    row += 1
workbook.close() 

In [ ]:

from openpyxl import load_workbook

workbook_name = 'sample.xlsx'
wb = load_workbook(workbook_name)
page = wb.active

# New data to write:
new_companies = [['name3','address3','tel3','web3'], ['name4','address4','tel4','web4']]

for info in new_companies:
    page.append(info)

wb.save(filename=workbook_name)

In [129]:
import openpyxl
wb = openpyxl.load_workbook('Data.xlsx')
ws = wb['CompanyCheck']
ws.append(['ID','company_name','company_type','company_status','incorporated_on','SIC_detail',
           'accounts','accounts_year','cash','net_worth','total_assets','total_liabilities'])

In [130]:
data = {'ID': [],
 'company_name': [],
 'company_type': [],
 'company_status': [],
 'incorporated_on': [],
 'SIC_detail': [],
 'accounts': [],
 'accounts_year': [],
 'cash': [],
 'net_worth': [],
 'total_assets': [],
 'total_liabilities': []}

for company in tqdm(companies):
    soup = get_soup(company)
    result = CompanyCheck(company, soup)
    content = list(result.values())
    ws.append(content)
    
#     for key in list(result.keys()):
#         data[key].append(result[key])
    
    time.sleep(2*random.randint(1,5))

Connection -->  500
Connection -->  404



In [131]:
wb.save(filename = 'Data.xlsx')
wb.close()

In [68]:
df = pd.DataFrame(data)

In [69]:
df.head()

,ID,company_name,company_type,company_status,incorporated_on,SIC_detail,accounts,accounts_year,cash,net_worth,total_assets,total_liabilities
0,01093301,AGEAS (UK) LIMITED,Private limited with Share Capital,Active - Accounts Filed,30 January 1973,64205\n Activities of financial...,Available to 31 Dec 2018. Next accounts due by...,2018,"£9,026,000.00","£679,502,000.00","£12,418,000.00","£1,058,000.00"
1,02341082,ASSURANT GENERAL INSURANCE LIMITED,Private limited with Share Capital,Active - Accounts Filed,30 January 1989,65120\n Non-life insurance\n ...,Available to 31 Dec 2018. Next accounts due by...,2018,"£12,379,000.00","£77,998,000.00","£63,443,000.00","£22,377,000.00"


In [133]:
soup = get_soup(companies[2])
result = CompanyCheck(companies[2], soup)
content = list(result.values())

In [134]:
content

[None, None, None, None, None, None, None, None, None, None, None, None]

In [135]:
companies[2]

'SE000103'